# Importing all moldules

In [20]:
import alpha_vantage
import pandas
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mpl_dates
import datetime

# CLASS ScriptData

In [21]:
class ScriptData(dict):

    global libray 
    libray = dict()

    def __getitem__(self,key):
        #print(type(lb[key]))
        #print(libray[key])
        return libray[key]


    def __setitem__(self, i, o):
        pass
    
    
    
    def __contains__(self, o):
        for key in libray.keys():
            if key == o:
                return True
            
        return False
    
    # __init__ function
    # def __init__(self):
    #     self = Userdict()


    def fetch_intraday_data(self,script):
        key = 'XTP4AMN2N5H9BL51'
        ts = TimeSeries(key,output_format='pandas')
        global data ,meta 
        data, meta = ts.get_intraday(script,interval='60min',outputsize='full')


    def convert_intraday_data(self,script):
        columns=['open','high','low','close','volumn']
        data.columns = columns
        data['timestamp']=data.index
        data.reset_index(drop=True, inplace=True)
        #data['timestamp']=data.index
        libray[script]=data
        #self['GOOGL'] = data
        #print(self[script])




    

In [22]:
script_data = ScriptData()

In [23]:
script_data.fetch_intraday_data('GOOGL')

In [24]:
script_data.convert_intraday_data('GOOGL')

In [25]:
script_data['GOOGL']

,open,high,low,close,volumn,timestamp
0,86.18,86.2000,85.89,86.1900,16965.0,2022-12-28 20:00:00
1,86.05,86.2200,86.04,86.0801,9283.0,2022-12-28 19:00:00
2,86.03,86.0985,85.98,86.0500,99064.0,2022-12-28 18:00:00
3,86.02,86.1600,85.92,86.0400,724680.0,2022-12-28 17:00:00
4,86.11,86.7499,85.94,86.0000,3739058.0,2022-12-28 16:00:00
...,...,...,...,...,...,...
612,90.72,90.9600,90.42,90.8300,149139.0,2022-11-02 09:00:00
613,90.79,90.8800,90.64,90.7900,38041.0,2022-11-02 08:00:00
614,90.70,90.7700,90.58,90.7200,24105.0,2022-11-02 07:00:00
615,91.05,91.0800,90.20,90.6700,23115.0,2022-11-02 06:00:00


In [26]:
script_data.fetch_intraday_data('AAPL')

In [27]:
script_data.convert_intraday_data('AAPL')

In [28]:
script_data['AAPL']

,open,high,low,close,volumn,timestamp
0,126.2600,126.3200,126.1600,126.3200,67042.0,2022-12-28 20:00:00
1,126.1400,126.2800,126.1400,126.2600,62647.0,2022-12-28 19:00:00
2,126.2200,126.9701,126.0400,126.1000,233823.0,2022-12-28 18:00:00
3,126.0400,126.3159,126.0000,126.2600,1942127.0,2022-12-28 17:00:00
4,126.8300,126.8400,125.8700,126.0500,15475315.0,2022-12-28 16:00:00
...,...,...,...,...,...,...
614,149.4623,149.5620,148.0165,148.4253,500256.0,2022-11-02 09:00:00
615,149.5122,149.5620,148.7843,148.9438,163320.0,2022-11-02 08:00:00
616,149.9010,149.9609,149.2529,149.3626,71444.0,2022-11-02 07:00:00
617,150.1603,150.1603,149.7116,149.9409,15923.0,2022-11-02 06:00:00


In [29]:
'GOOGL' in script_data

True

In [30]:
'AAPL' in script_data 

True

In [31]:
'UNCLE' in script_data

False

# Indicator method


In [32]:
def indicator1(df,timeperiod=10):
    df1 = pandas.DataFrame()
    df1['indicator'] = df['close'].rolling(timeperiod).mean()
    return df1

In [33]:
indicator1(script_data['GOOGL'], timeperiod=5)

,indicator
0,NaN
1,NaN
2,NaN
3,NaN
4,86.07202
...,...
612,89.85148
613,90.12548
614,90.41300
615,90.60600


In [34]:
indicator1(script_data['AAPL'], 5)

,indicator
0,NaN
1,NaN
2,NaN
3,NaN
4,126.19800
...,...
614,148.55788
615,148.59178
616,148.76834
617,148.98370


In [35]:

class Strategy(ScriptData):
    
    def __init__(self,script):
        self.script = script


    def get_script_data(self):
        self.fetch_intraday_data(self.script)
        self.convert_intraday_data(self.script)
        global df
        global indicator_data
        df = data['close'].to_frame()
        df['timestamp'] = data['timestamp']
        df['indicator_data'] = indicator1(df,5)

        

    def get_signals(self):
        
        signals = df
        signals['signal'] = np.nan
        for i in range(len(df)):
            if df.iloc[i-1]['close']>df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] < df.iloc[i]['indicator_data']:
                signals['signal'][i] = 'BUY'
            elif df.iloc[i-1]['close'] < df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] > df.iloc[i]['indicator_data']:
                signals['signal'][i] = 'SELL'

        return signals[['timestamp','signal']]
        
        
    def plot(self):
        df2 = df['timestamp'].to_frame()
        df2['df_close'] = df['close']            
        df2['indicator_data']= indicator_data['indicator']
        plt.plot(df2['timestamp'],df2['df_close'],color='red')
        plt.plot(df2['timestamp'],df2['indicator_data'],color='navy')
        
    
    
        
        
        
         
            
            
        
        
        


In [36]:
strategy = Strategy('NVDA')
strategy.get_script_data()

In [37]:
strategy.get_signals()

,timestamp,signal
0,2022-12-28 20:00:00,NaN
1,2022-12-28 19:00:00,NaN
2,2022-12-28 18:00:00,NaN
3,2022-12-28 17:00:00,NaN
4,2022-12-28 16:00:00,NaN
...,...,...
614,2022-11-02 09:00:00,BUY
615,2022-11-02 08:00:00,NaN
616,2022-11-02 07:00:00,NaN
617,2022-11-02 06:00:00,NaN


In [39]:
#strategy.plot()

NameError: name 'indicator_data' is not defined